In [59]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
np.random.seed(42)

N_ESTIMATORS = 4
MAX_DEPTH = 15
SUBSPACE_DIM = 2


class sample(object):
    def __init__(self, X, n_subspace):
        self.idx_subspace: np.ndarray = self.random_subspace(X, n_subspace)

    def __call__(self, X, y):
        idx_obj = self.bootstrap_sample(X)
        X_sampled, y_sampled = self.get_subsample(
            X, y, self.idx_subspace, idx_obj)
        return X_sampled, y_sampled

    @staticmethod
    def bootstrap_sample(X, random_state=42):
        """
        Заполните тело этой функции таким образом, чтобы она возвращала массив индексов выбранных при помощи бэггинга индексов.
        Пользуйтесь только инструментами, реализованными в numpy.random, выставляя везде, где это необходимо, random_state=42
        """
        arr = np.random.choice(len(X), size=len(X), replace=True)

        uniq_idxes = np.unique(arr)
        return uniq_idxes

    @staticmethod
    def random_subspace(X, n_subspace, random_state=42):
        first_elem = X[0]
        idxes = np.random.choice(
            len(first_elem), size=n_subspace, replace=False)
        return idxes

    @staticmethod
    def get_subsample(X, y, idx_subspace, idx_obj):
        """
        Заполните тело этой функции таким образом, чтобы она возвращала подвыборку x_sampled, y_sampled
        по значениям индексов признаков(idx_subspace) и объектов(idx_obj) , которые должны в неё попасть
        """

        return X[np.ix_(idx_obj, idx_subspace)], y[idx_obj]


class random_forest(object):
    def __init__(self, n_estimators: int, max_depth: int, subspaces_dim: int, random_state: int):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.subspaces_dim = subspaces_dim
        self.random_state = random_state
        self._estimators: list[tuple(DecisionTreeClassifier, np.ndarray)] = []

        """
      Задайте все необходимые поля в рамках конструктора класса
    """

    def fit(self, X, y):
        for i in range(self.n_estimators):
            selecter = sample(X, self.subspaces_dim)

            model = DecisionTreeClassifier(
                max_depth=self.max_depth, random_state=self.random_state)

            X_sample, y_sample = selecter.get_subsample(
                X, y, selecter.idx_subspace, selecter.bootstrap_sample(X))

            model.fit(X_sample, y_sample)
            self._estimators.append((model, selecter.idx_subspace))

    def predict(self, X):
        res = []
        for i in range(self.n_estimators):
            res.append(self._estimators[i][0].predict(X[:,self._estimators[i][1]]))
        res=np.array(res)
        most_popular_elements = []
        for col in range(res.shape[1]):
            column_values = res[:, col]
            mode = np.argmax(np.bincount(column_values))
            most_popular_elements.append(mode)
        
        
            
        return np.array(most_popular_elements)

In [28]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [29]:
iris = load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [64]:
tree = random_forest(4, 15, 2, 42)
tree.fit(X_train, y_train)

In [65]:
from sklearn.metrics import accuracy_score


preds = tree.predict(X_test)
preds
accuracy_score(preds,y_test)

1.0